# AiiDAlab QauntumESPRESSO App

## Job List


In [ ]:
import ipywidgets as ipw
from aiida import load_profile
from aiida.tools.query.calculation import CalculationQueryBuilder

load_profile()


process_label = "QeAppWorkChain"
projections = ["pk", "ctime", "state", "label"]


def query_data():
    builder = CalculationQueryBuilder()
    filters = builder.get_filters(
        process_label=process_label,
    )
    query_set = builder.get_query_set(
        filters=filters,
        order_by={"ctime": "desc"},
    )
    projected = builder.get_projected(
        query_set,
        projections=projections,
    )
    return projected


table = ipw.HTML()

table.value = """
        <style>
            .df { border: none; }
            .df tbody tr:nth-child(odd) { background-color: #e5e7e9; }
            .df tbody tr:nth-child(odd):hover { background-color:   #f5b7b1; }
            .df tbody tr:nth-child(even):hover { background-color:  #f5b7b1; }
            .df tbody td { min-width: 80px; text-align: center; border: none }
            .df th { text-align: center; border: none;  border-bottom: 1px solid black;}
        </style>
        """

# use pandas to create a dataframe from the query data
import pandas as pd

df = pd.DataFrame(query_data()[1:], columns=projections)

# add a column to the dataframe with a button to delete the archive
for index, row in df.iterrows():
    pk = row['pk']
    df.at[index, "formula"] = row['label'].split("structure")[-0]
    df.at[index, "relaxed"] = "is relaxed" in row['label']
    df.at[index, "properties"] = row['label'].split("properties on")[-1]
    df.at[index, 'Delete'] = f"""<a href="./delete.ipynb?pk={pk}" target="_blank">Delete</a>"""
    df.at[index, 'Inspect'] = f"""<a href="./qe.ipynb?pk={pk}" target="_blank">Inspect</a>"""

# delete the label column
df = df.drop(columns=['label'])

table.value += df.to_html(classes="df", escape=False, index=False)
table